In [ ]:
import pandas as pd

# Load the datasets
contractor_registry_path = "../data/raw/Contractor_Registry_Certificate_20250215.csv"


df_contractors = pd.read_csv(contractor_registry_path)




In [20]:
print(f"Total records in Registry: {len(df_contractors)}")

Total records in Registry: 7017


In [21]:
df_contractors.head()

,Certificate Number,Business Name,DBA Name,Business Type,Business is MWBE Owned,Business is Publicly Traded,Business Officers,Address,Address 2,City,...,Debarment Start Date,Debarment End Date,Business has final determination for violation of Labor or Tax Law,Business has final determination safety standard violations,Business is associated with an apprenticeship program,Business is sponsor of a program,Business is signatory to a group program,Business has Workers Compensation Insurance,Business is exempt from Workers Compensation Insurance,Georeference
0,24-639SQ-CR,10100 Inc.,"10100, Inc.",Corporation,No,No,NaN,955 West River Road,NaN,Grand Island,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-78.99049 42.97805)
1,24-63ZE1-CR,"1068 Curry Road, Inc.",city glass company,Corporation,No,No,NaN,1068 curry rd,NaN,"schenectady, ny 12306",...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-73.969 42.78614)
2,25-64TYL-CR,11400 LLC,CLARK FOOD SERVICE EQUIPMENT,LLC,No,No,NaN,2551 HORSESHOE RD,NaN,LANCASTER,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-76.206 40.07246)
3,25-6414J-CR,"1349 Main, Gorenflo's Buffalo Wholesale Lock C...",Gorenflo's Buffalo Wholesale Lock,Corporation,No,No,NaN,1349 Main Street,NaN,Buffalo,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-78.86594 42.91033)
4,25-64V75-CR,1895 ELECTRIC LLC,NaN,LLC,No,No,NaN,60 SCHOL ST#114,NaN,ORCHARD PARK,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-78.74632 42.76559)


In [22]:
# Check for missing values
print("\nMissing Values:\n", df_contractors.isnull().sum()[df_contractors.isnull().sum() > 0])

# Check data types
print("\nData Types:\n", df_contractors.dtypes)


Missing Values:
 DBA Name                                                                 5495
Business Type                                                              36
Business Officers                                                        6921
Address 2                                                                6062
State                                                                       4
Phone                                                                       4
Reason business does not have a NYS DOL Employer Registration Number     6331
Debarment State or Federal Law                                           6997
Debarment State                                                          6999
Debarment Start Date                                                     6997
Debarment End Date                                                       6997
Georeference                                                              651
dtype: int64

Data Types:
 Certificate Number 

In [23]:
df_contractors["Phone"] = df_contractors["Phone"].fillna("Unknown")


In [24]:
#fill missing values
df_contractors["Business Officers"].fillna("None Listed", inplace=True)

In [25]:
import re

# Define the cleaning function
def clean_business_name(name):
    """Cleans and standardizes business and DBA names without removing suffixes."""
    if pd.isna(name) or name.strip() == "":  # Handle NaN and empty values
        return "No DBA"
    name = name.upper().strip()  # Convert to uppercase and remove leading/trailing spaces
    name = re.sub(r"[^\w\s&]", "", name)  # Remove punctuation except '&'
    name = re.sub(r"\s+", " ", name)  # Replace multiple spaces with a single space
    return name.strip()

# Apply cleaning function to DBA Name and Business Name columns
df_contractors["DBA Name"] = df_contractors["DBA Name"].apply(clean_business_name)
df_contractors["Business Name"] = df_contractors["Business Name"].apply(clean_business_name)

# Standardize DBA Name: Fill missing or blank values with "No DBA"
df_contractors["DBA Name"] = df_contractors["DBA Name"].fillna("No DBA")
df_contractors.loc[df_contractors["DBA Name"].str.strip() == "", "DBA Name"] = "No DBA"

In [26]:
# Ensure all relevant columns are strings before processing
df_contractors["Address"] = df_contractors["Address"].astype(str)
df_contractors["Address 2"] = df_contractors["Address 2"].astype(str)
df_contractors["City"] = df_contractors["City"].astype(str)
df_contractors["State"] = df_contractors["State"].astype(str)
df_contractors["Zip Code"] = df_contractors["Zip Code"].astype(str)

# Regex pattern: Moves unit/floor-related terms, but keeps "PO BOX" in Address
suite_pattern = r"\b(\d{1,4}(?:ST|ND|RD|TH)?\s*(?:FLOOR|LEVEL|ROOM|UNIT|SUITE|APT|BLDG|PH|PENTHOUSE|OFFICE|WING|#|LOT|DEPT|BUILDING))\b"

# List to track modified rows
modified_rows = []

# Function to clean addresses and move suite/unit numbers to Address 2
def clean_address_row(index, row):
    address, address2, city, state, zip_code = (
        str(row["Address"]).strip(),
        str(row["Address 2"]).strip(),
        str(row["City"]).strip(),
        str(row["State"]).strip(),
        str(row["Zip Code"]).strip(),
    )

    # Preserve "PO BOX" in Address, only move other suite/unit numbers
    if "PO BOX" in address.upper():
        keep_po_box = True
    else:
        keep_po_box = False

    # Search for suite/unit numbers in the Address field (excluding PO BOX)
    suite_match = re.search(suite_pattern, address, re.IGNORECASE)
    
    if suite_match:
        suite_part = suite_match.group(0).upper()  # Extract full suite/floor phrase
        address = re.sub(suite_pattern, "", address, flags=re.IGNORECASE).strip()  # Remove from Address

        # Ensure "PO BOX" remains in Address
        if keep_po_box:
            address = address.replace("PO BOX", "PO BOX").strip()

        # Move to Address 2, preserving existing values
        if address2.lower() in ["not applicable", "nan", "na", ""]:
            address2 = suite_part
        else:
            address2 = f"{address2} {suite_part}".strip()

        modified_rows.append(index)  # Track modified rows

    return pd.Series([address.upper(), address2.upper(), city.title(), state.upper(), zip_code.zfill(5)])

# Apply function to clean all rows
df_contractors[["Address", "Address 2", "City", "State", "Zip Code"]] = df_contractors.apply(
    lambda row: clean_address_row(row.name, row), axis=1
)

# Standardize formatting
df_contractors["City"] = df_contractors["City"].str.upper().str.replace(r"[,0-9]+", "", regex=True)
df_contractors["State"] = df_contractors["State"].str.upper().str.strip()
df_contractors["Address"] = df_contractors["Address"].str.replace(r"\s+", " ", regex=True).str.strip()
df_contractors["Address 2"] = df_contractors["Address 2"].str.replace(r"\s+", " ", regex=True).str.strip()
df_contractors["Zip Code"] = df_contractors["Zip Code"].astype(str).str.zfill(5)  # Ensure ZIP codes are 5 digits

# Display modified rows for verification
print(f"Modified {len(modified_rows)} rows.")
print(df_contractors.loc[modified_rows, ["Address", "Address 2", "City", "State", "Zip Code"]].sample(10))


Modified 34 rows.
                       Address   Address 2           City State Zip Code
670            40 WALL STREET,  11TH FLOOR       NEW YORK    NY    10005
3438           30 WALL STREET,   8TH FLOOR       NEW YORK    NY    10005
3783         368 NINTH AVENUE,   8TH FLOOR       NEW YORK    NY    10001
1018  67 WEST STREET, SUITE B2       401 #       BROOKLYN    NY    11222
3629                 1352 NY 1        40 #   SCHAGHTICOKE    NY    12154
5495        312 FIFTH AVENUE -   4TH FLOOR       NEW YORK    NY    10001
1808                      ROAD   3329 WING      JAMESTOWN    NY    14701
2260          17 STATE STREET,  36TH FLOOR       NEW YORK    NY    10004
1766           57 BEACH STREET   2ND FLOOR  STATEN ISLAND    NY    10304
5813   389 INTERPACE PARKWAY -   5TH FLOOR     PARSIPPANY    NJ    07054


In [11]:
# Fill missing or blank "Address 2" values with "Not Applicable"
df_contractors["Address 2"] = df_contractors["Address 2"].fillna("Not Applicable")
df_contractors.loc[df_contractors["Address 2"].str.strip() == "", "Address 2"] = "Not Applicable"



In [12]:
# Standardize "Business is Debarred" column
df_contractors["Business has been debarred"] = df_contractors["Business has been debarred"].str.upper().str.strip()

# Fill in Debarment Start Date and Debarment End Date if "Business is Debarred" is "NO"
df_contractors.loc[
    df_contractors["Business has been debarred"] == "NO", 
    ["Debarment Start Date", "Debarment End Date"]
] = "Not Applicable"



In [13]:
# Fill missing "Debarment State or Federal Law" with "Not Applicable"
df_contractors["Debarment State or Federal Law"].fillna("Not Applicable", inplace=True)

# Fill missing "Debarment State" with "Not Applicable"
df_contractors["Debarment State"].fillna("Not Applicable", inplace=True)

# If "Debarment State or Federal Law" mentions "Federal", set "Debarment State" to "Federal Debarment"
df_contractors.loc[
    df_contractors["Debarment State or Federal Law"].str.contains("Federal", case=False, na=False), 
    "Debarment State"
] = "Federal Debarment"

# Verify changes
print("Updated 'Debarment State or Federal Law' and 'Debarment State' fields.")
print(df_contractors[["Debarment State or Federal Law", "Debarment State"]].head(10))


Updated 'Debarment State or Federal Law' and 'Debarment State' fields.
  Debarment State or Federal Law Debarment State
0                 Not Applicable  Not Applicable
1                 Not Applicable  Not Applicable
2                 Not Applicable  Not Applicable
3                 Not Applicable  Not Applicable
4                 Not Applicable  Not Applicable
5                 Not Applicable  Not Applicable
6                 Not Applicable  Not Applicable
7                 Not Applicable  Not Applicable
8                 Not Applicable  Not Applicable
9                 Not Applicable  Not Applicable


In [14]:
df_contractors.columns = df_contractors.columns.str.strip()

In [15]:
df_contractors.loc[
    df_contractors["Business has a NYS DOL Employer Registration Number"] == "Yes",
    "Reason business does not have a NYS DOL Employer Registration Number"
] = "Not Applicable"

In [28]:
import pandas as pd
# Load the datasets
cleaned_data_path = "cleaned_data.csv"


df_cleaned = pd.read_csv(cleaned_data_path)



In [29]:
df_cleaned.head()

,Certificate Number,Business Name,DBA Name,Business Type,Business is MWBE Owned,Business is Publicly Traded,Business Officers,Address,Address 2,City,...,Debarment Start Date,Debarment End Date,Business has final determination for violation of Labor or Tax Law,Business has final determination safety standard violations,Business is associated with an apprenticeship program,Business is sponsor of a program,Business is signatory to a group program,Business has Workers Compensation Insurance,Business is exempt from Workers Compensation Insurance,Georeference
0,24-639SQ-CR,10100 INC,10100 INC,Corporation,No,No,None Listed,955 WEST RIV RD,Not Applicable,GRAND ISLAND,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-78.99049 42.97805)
1,24-63ZE1-CR,1068 CURRY ROAD INC,CITY GLASS COMPANY,Corporation,No,No,None Listed,1068 CURRY RD,Not Applicable,SCHENECTADY,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-73.969 42.78614)
2,25-64TYL-CR,11400 LLC,CLARK FOOD SERVICE EQUIPMENT,LLC,No,No,None Listed,2551 HORSESHOE RD,Not Applicable,LANCASTER,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-76.206 40.07246)
3,25-6414J-CR,1349 MAIN GORENFLOS BUFFALO WHOLESALE LOCK CO INC,GORENFLOS BUFFALO WHOLESALE LOCK,Corporation,No,No,None Listed,1349 MAIN ST,Not Applicable,BUFFALO,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-78.86594 42.91033)
4,25-64V75-CR,1895 ELECTRIC LLC,No DBA,LLC,No,No,None Listed,60 SCHOL ST#114,Not Applicable,ORCHARD PARK,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-78.74632 42.76559)


In [30]:
import string


# Function to clean text: uppercase, remove punctuation, strip spaces
def clean_text(text):
    if pd.isna(text):  # Handle NaN values
        return text
    text = text.upper().strip()  # Convert to uppercase and strip spaces
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    return text

# Apply cleaning to Business Name and Address columns
df_cleaned["Business Name"] = df_cleaned["Business Name"].apply(clean_text)
df_cleaned["DBA Name"] = df_cleaned["DBA Name"].apply(clean_text)  # If you want DBA Name too
df_cleaned["Address"] = df_cleaned["Address"].apply(clean_text)
df_cleaned["Address 2"] = df_cleaned["Address 2"].apply(clean_text)  # If Address 2 exists

# Save the cleaned file
df_cleaned.to_csv("cleaned_data.csv", index=False)

print("✅ Business Names and Addresses have been cleaned (Uppercase + No Punctuation).")


✅ Business Names and Addresses have been cleaned (Uppercase + No Punctuation).


In [31]:
df_cleaned.head()

,Certificate Number,Business Name,DBA Name,Business Type,Business is MWBE Owned,Business is Publicly Traded,Business Officers,Address,Address 2,City,...,Debarment Start Date,Debarment End Date,Business has final determination for violation of Labor or Tax Law,Business has final determination safety standard violations,Business is associated with an apprenticeship program,Business is sponsor of a program,Business is signatory to a group program,Business has Workers Compensation Insurance,Business is exempt from Workers Compensation Insurance,Georeference
0,24-639SQ-CR,10100 INC,10100 INC,Corporation,No,No,None Listed,955 WEST RIV RD,NOT APPLICABLE,GRAND ISLAND,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-78.99049 42.97805)
1,24-63ZE1-CR,1068 CURRY ROAD INC,CITY GLASS COMPANY,Corporation,No,No,None Listed,1068 CURRY RD,NOT APPLICABLE,SCHENECTADY,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-73.969 42.78614)
2,25-64TYL-CR,11400 LLC,CLARK FOOD SERVICE EQUIPMENT,LLC,No,No,None Listed,2551 HORSESHOE RD,NOT APPLICABLE,LANCASTER,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-76.206 40.07246)
3,25-6414J-CR,1349 MAIN GORENFLOS BUFFALO WHOLESALE LOCK CO INC,GORENFLOS BUFFALO WHOLESALE LOCK,Corporation,No,No,None Listed,1349 MAIN ST,NOT APPLICABLE,BUFFALO,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-78.86594 42.91033)
4,25-64V75-CR,1895 ELECTRIC LLC,NO DBA,LLC,No,No,None Listed,60 SCHOL ST114,NOT APPLICABLE,ORCHARD PARK,...,Not Applicable,Not Applicable,No,No,No,No,No,Yes,No,POINT (-78.74632 42.76559)


In [32]:
# Ensure State abbreviations are uppercase for consistency
df_cleaned["State"] = df_cleaned["State"].str.upper()
df_cleaned["City"] = df_cleaned["City"].str.upper().str.strip()  # Standardize city format

# Define the set of U.S. state abbreviations
us_states = set([
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY",
    "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND",
    "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
])

# Function to check if City contains a state abbreviation (e.g., "Buffalo NY")
def city_contains_state(row):
    city = row["City"].strip()
    
    # Check if city contains a state abbreviation at the end (case insensitive)
    match = re.search(rf"\b({'|'.join(us_states)})\b$", city, re.IGNORECASE)
    
    if match:
        return match.group(0).upper()  # Return the state abbreviation found
    return None

# Apply the function to flag incorrect city names and extract the incorrect state
df_cleaned["City Contains State"] = df_cleaned.apply(city_contains_state, axis=1)

# Filter only the incorrect cases
incorrect_cities = df_cleaned[df_cleaned["City Contains State"].notna()]

# Display incorrect city names before fixing
print(f"Found {len(incorrect_cities)} incorrect 'City' values containing state abbreviations:")
print(incorrect_cities[["City", "State", "Zip Code"]].head(10))

# Function to clean city names by removing the state abbreviation
def fix_city(row):
    if pd.notna(row["City Contains State"]):
        state_from_city = row["City Contains State"]
        new_city = re.sub(rf"\b{state_from_city}\b$", "", row["City"], flags=re.IGNORECASE).strip()  # Remove state from City
        return new_city.title()  # Corrected City name
    return row["City"]  # No change needed

# Apply the fix without moving the state
df_cleaned["City"] = df_cleaned.apply(fix_city, axis=1)

# Drop the temporary "City Contains State" column
df_cleaned.drop(columns=["City Contains State"], inplace=True)

# Verify the fix
print("\n✅ Updated City column (State remains unchanged):")
print(df_cleaned[["City", "State", "Zip Code"]].sample(10))


Found 0 incorrect 'City' values containing state abbreviations:
Empty DataFrame
Columns: [City, State, Zip Code]
Index: []

✅ Updated City column (State remains unchanged):
                City State Zip Code
3146       LYNDHURST    NJ    07071
284        MT VERNON    NY    10550
5544          HILTON    NY    14468
5906         HAMBURG    NY    14075
1536            ELMA    NY    14059
5748          DALTON    GA    30722
6037   EAST SYRACUSE    NY    13057
3180   STATEN ISLAND    NY    10309
1881           UTICA    NY    13501
5307  QUEENS VILLAGE    NY    11428


In [ ]:
# Save the updated DataFrame to a CSV file
output_file_path = "../data/processed/cleaned_contractors.csv"  # Update path as needed
df_cleaned.to_csv(output_file_path, index=False)